In [2]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [3]:
querys={

    'Estoque':

    """
    
    SELECT * FROM netfeira.vw_estoque
    
    """,

    'Vendas':

    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME, @DTINICIO DATETIME,@DIAS SMALLINT

    SET @DIAS=30

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=DATEADD(DAY,-1,@DTBASE)

    SET @DTINICIO=DATEADD(DAY,@DIAS*-1,@DTFIM)

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]<>'OUTROS'

    
    """
}

In [4]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [5]:
tabelas_df['Estoque'].columns

Index(['Local', 'Tipo', 'SKU', 'Cód. Fabricante', 'Produto', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Fator CX', 'Unid. CMP',
       'Estoque', 'Qtde Pendente', 'Qtde Disponível', 'Caixa Disponível',
       'Peso Disponível', 'Qtde CMP'],
      dtype='object')

In [6]:
tabelas_df['Vendas'].columns

Index(['Origem', 'Situação Entrega', 'Usuário', 'Equipe Usuário',
       'Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [7]:
def Classificao(perc):
    
    if(perc<=80):
        
        retorno='A'
        
        pass
    
    elif(perc<=95):
        
        retorno='B'
        
        pass
    
    else:
        
        retorno='C'
        
        pass
    
    return retorno
    
    pass

In [8]:
vendas_df=pd.DataFrame()

vendas_df=tabelas_df['Vendas'][['SKU','Qtde']].groupby(['SKU'],as_index=False).sum()

vendas_df.sort_values('Qtde',ascending=False,ignore_index=True,inplace=True)

qtde=vendas_df['Qtde'].sum()

vendas_df['Qtde Empresa']=qtde

vendas_df=vendas_df.loc[vendas_df['Qtde']>0]

res=0

valores=[]

for indice,linha in vendas_df.iterrows():
    
    res+=linha['Qtde'].sum()
    
    valores.append(res)
    
    pass

vendas_df['Qtde Acumulado']=valores

vendas_df['Perc']=vendas_df.apply(lambda info: round(info['Qtde Acumulado']/info['Qtde Empresa'],4)*100,axis=1)

vendas_df['Classificação']=vendas_df['Perc'].apply(Classificao)

vendas_df

,SKU,Qtde,Qtde Empresa,Qtde Acumulado,Perc,Classificação
0,1113,13025.0,195752.9328,13025.0000,6.65,A
1,9549,7768.0,195752.9328,20793.0000,10.62,A
2,1106,6578.0,195752.9328,27371.0000,13.98,A
3,594,5829.0,195752.9328,33200.0000,16.96,A
4,9337,5576.0,195752.9328,38776.0000,19.81,A
...,...,...,...,...,...,...
336,9344,1.0,195752.9328,195748.9328,100.00,C
337,719,1.0,195752.9328,195749.9328,100.00,C
338,463,1.0,195752.9328,195750.9328,100.00,C
339,473,1.0,195752.9328,195751.9328,100.00,C


In [9]:
estoque_df=pd.DataFrame()

estoque_df=tabelas_df['Estoque'].merge(vendas_df,on='SKU',how='left')[['Local', 'Tipo', 'SKU', 'Cód. Fabricante', 'Produto', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Fator CX', 'Unid. CMP',
       'Qtde Disponível', 'Caixa Disponível', 'Peso Disponível', 'Qtde CMP','Classificação','Perc']]

In [10]:
local_df=pd.DataFrame()

local_df=estoque_df[['Local','Qtde Disponível']].groupby(['Local'],as_index=False).sum()

local_df=local_df.loc[local_df['Qtde Disponível']>0]

locais=local_df['Local'].tolist()

In [11]:
estoque_df=estoque_df.loc[estoque_df['Local'].isin(locais)]

In [12]:
estoque_df.columns

Index(['Local', 'Tipo', 'SKU', 'Cód. Fabricante', 'Produto', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Fator CX', 'Unid. CMP',
       'Qtde Disponível', 'Caixa Disponível', 'Peso Disponível', 'Qtde CMP',
       'Classificação', 'Perc'],
      dtype='object')

In [13]:
estoque_df=estoque_df[['SKU', 'Cód. Fabricante', 'Produto', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha','Classificação','Perc','Qtde Disponível']].groupby(['SKU', 'Cód. Fabricante', 'Produto', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha','Classificação','Perc'],as_index=False).sum()

In [14]:
qtde_prod=len(estoque_df['SKU'].unique().tolist())

qtde_prod

338

In [15]:
len(estoque_df['SKU'].loc[estoque_df['Classificação']=='A'].unique().tolist())

74

In [16]:
len(estoque_df['SKU'].loc[estoque_df['Classificação']=='B'].unique().tolist())

68

In [17]:
len(estoque_df['SKU'].loc[estoque_df['Classificação']=='C'].unique().tolist())

196

In [18]:
classificacao_df=pd.DataFrame()

classificacao_df=estoque_df[['Classificação','SKU']].groupby(['Classificação'],as_index=False).count()

classificacao_df['Contagem']=qtde_prod

classificacao_df['Perc']=classificacao_df.apply(lambda info: round(info['SKU']/info['Contagem'],4)*100,axis=1)

classificacao_df

,Classificação,SKU,Contagem,Perc
0,A,74,338,21.89
1,B,68,338,20.12
2,C,196,338,57.99


In [31]:
df=estoque_df[['SKU','Produto','Perc']].loc[(estoque_df['Classificação']=='A')&(estoque_df['Qtde Disponível']<=0)]

df.sort_values('Perc',ascending=False,ignore_index=True,inplace=True)

df

,SKU,Produto,Perc
0,95,"IQF FRAMBOESA CONGELADOS 1,200 KG",73.44
1,9566,"CONGELADO BROCOLIS DE MARCHI FG ( 1,2KG )",45.16
2,9568,SUCO PINK LEMONADE 300ML DEMARCHI,22.33


In [20]:
tabelas_df['Estoque']['Qtde Pendente'].loc[tabelas_df['Estoque']['Local'].isin(locais)].sum()

2745.0

In [21]:
len(estoque_df['SKU'].unique().tolist())

338

In [22]:
prod_disp=len(estoque_df['SKU'].loc[estoque_df['Qtde Disponível']<=0].unique().tolist())

In [26]:
prod_ativo=qtde_prod-prod_disp

In [27]:
perc=round(prod_ativo/qtde_prod,4)*100

In [28]:
perc

84.32

In [25]:
len(estoque_df['SKU'].loc[(estoque_df['Classificação']=='A')&(estoque_df['Qtde Disponível']<=0)].unique().tolist())

3